La idea del spline cúbico es en vez de hacer una función que me de todos los puntos, lo que hacemos es hacer n funciones entre punto y punto, y las tratamos de hacer con una interpolación de una función cúbica. 

Inspirado en este video: https://youtu.be/yOUst2672qo

In [51]:
using DataFrames

In [75]:
# Path: clase6/splines.ipynb
function build_first_order_continuity_coefficients(X, points)

    """
        Given a matrix full of zeros, and of size (4n, 4n), where n is the number of polynomials to be found
        for the cubic spline, this function fills the upper part of the matrix corresponding to the equations 
        that say that the polynomials must be continuous at the intersections of the intervals.

        Arguments:
            X: Matrix of zeros of size (4n, 4n)
            points: Vector of points [x,y] to be interpolated
    """

    # Check that the matrix is square
    size(X, 1) == size(X, 2) || error("The matrix must be square")

    row = 1
    n_intervals = length(points) - 1

    # Fill the upper part of the matrix
    for i in 1:n_intervals
        
        # Columns to be filled
        cols = (4 * i - 3):(4 * i)
        
        # Points to get the coefficients to fill the columns with
        start_point = points[i]
        end_point = points[i + 1]

        X[row, cols] = [start_point[1]^3, start_point[1]^2, start_point[1][1], 1]
        X[row+1, cols] = [end_point[1]^3, end_point[1]^2, end_point[1], 1]
        row += 2
    end
    

    return X
end

function build_second_derivative_continuity_coefficients(X, points)

    """
        Given a matrix full of zeros, and of size (4n, 4n), where n is the number of polynomials to be found
        for the cubic spline, this function fills the middle part of the matrix corresponding to the equations 
        that say that the first derivative of the polynomials must be continuous at the intersections of the intervals, 
        this is that the first derivative of the polynomial at the end of an interval must be equal to the first derivative
        of the polynomial at the beginning of the next interval.

        Arguments:
            X: Matrix of zeros of size (4n, 4n)
            points: Vector of points [x,y] to be interpolated
    """

    # Check that the matrix is square
    size(X, 1) == size(X, 2) || error("The matrix must be square")

    n_intervals = length(points) - 2
    row = (n_intervals+2) * 2 - 1

    # Fill the upper part of the matrix
    for i in 1:n_intervals
        
        # Columns to be filled
        col1 = (4 * i - 3):(4 * i)
        col2 = (4 * i + 1):(4 * i + 4)

        # Points to get the coefficients to fill the columns with
        start_point = points[i]
        end_point = points[i + 1]

        X[row, col1] = [-3*start_point[1]^2, -2*start_point[1], -1, 0]
        X[row, col2] = [ 3*end_point[1]^2,    2*end_point[1],    1, 0]

        row += 1
    end
    

    return X
end

build_second_order_continuity_coefficients (generic function with 1 method)

In [76]:
#list of points to interpolate
points = [[3, 1], [1, 1], [2, 0.5], [5, 0.69]]

n_intervals = length(points) - 1

#matrix of zeros of size (4n, 4n)
X = zeros(4 * n_intervals, 4 * n_intervals)

#fill the upper part of the matrix and print the complete matrix
X = build_first_order_continuity_coefficients(X, points)
X = build_second_derivative_continuity_coefficients(X, points)
#DataFrame(X, :auto)
X

12×12 Matrix{Float64}:
  27.0   9.0   3.0  1.0   0.0   0.0   0.0  0.0    0.0   0.0  0.0  0.0
   1.0   1.0   1.0  1.0   0.0   0.0   0.0  0.0    0.0   0.0  0.0  0.0
   0.0   0.0   0.0  0.0   1.0   1.0   1.0  1.0    0.0   0.0  0.0  0.0
   0.0   0.0   0.0  0.0   8.0   4.0   2.0  1.0    0.0   0.0  0.0  0.0
   0.0   0.0   0.0  0.0   0.0   0.0   0.0  0.0    8.0   4.0  2.0  1.0
   0.0   0.0   0.0  0.0   0.0   0.0   0.0  0.0  125.0  25.0  5.0  1.0
 -27.0  -6.0  -1.0  0.0   3.0   2.0   1.0  0.0    0.0   0.0  0.0  0.0
   0.0   0.0   0.0  0.0  -3.0  -2.0  -1.0  0.0   12.0   4.0  1.0  0.0
   0.0   0.0   0.0  0.0   0.0   0.0   0.0  0.0    0.0   0.0  0.0  0.0
   0.0   0.0   0.0  0.0   0.0   0.0   0.0  0.0    0.0   0.0  0.0  0.0
   0.0   0.0   0.0  0.0   0.0   0.0   0.0  0.0    0.0   0.0  0.0  0.0
   0.0   0.0   0.0  0.0   0.0   0.0   0.0  0.0    0.0   0.0  0.0  0.0

In [22]:
4*1 - 3:4*1

1:4